In [13]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler , TensorDataset
from transformers import BertTokenizer, BertModel, BertConfig , BertForSequenceClassification , AdamW
import re
# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()
import torch
from tqdm import tqdm

In [16]:
import torch
from transformers import  XLNetTokenizer, XLNetForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import sklearn.metrics as sk
import re
import collections
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler , TensorDataset
from transformers import BertTokenizer, BertModel, BertConfig , BertForSequenceClassification , AdamW
from tqdm import tqdm
from torch import cuda
from sklearn.model_selection import train_test_split

c:\Users\Ashish Paide\anaconda3\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
# Step 1: Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming binary classification



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
def load_data(filename='./data/imdb.train'):
    '''
    :param filename: the system location of the data to load
    :return: the text (x) and its label (y)
             the text is a list of words and is not processed
    '''

    # stop words taken from nltk
    stop_words = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours',
                  'yourself','yourselves','he','him','his','himself','she','her','hers','herself',
                  'it','its','itself','they','them','their','theirs','themselves','what','which',
                  'who','whom','this','that','these','those','am','is','are','was','were','be',
                  'been','being','have','has','had','having','do','does','did','doing','a','an',
                  'the','and','but','if','or','because','as','until','while','of','at','by','for',
                  'with','about','against','between','into','through','during','before','after',
                  'above','below','to','from','up','down','in','out','on','off','over','under',
                  'again','further','then','once','here','there','when','where','why','how','all',
                  'any','both','each','few','more','most','other','some','such','no','nor','not',
                  'only','own','same','so','than','too','very','s','t','can','will','just','don',
                  'should','now','d','ll','m','o','re','ve','y','ain','aren','couldn','didn',
                  'doesn','hadn','hasn','haven','isn','ma','mightn','mustn','needn','shan',
                  'shouldn','wasn','weren','won','wouldn']

    x, y = [], []
    with open(filename, "r") as f:
        for line in f:
            line = re.sub(r'\W+', ' ', line).strip().lower()  # perhaps don't make words lowercase?
            x.append(line[:-1])
            x[-1] = ' '.join(word for word in x[-1].split() if word not in stop_words)
            y.append(line[-1])
    return x, np.array(y, dtype=int)

In [10]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 100
VALID_BATCH_SIZE = 100
EPOCHS = 15
LEARNING_RATE = 1e-05




In [20]:
# Load your sentiment analysis dataset and preprocess it
X_train_texts, Y_train = load_data('./data/CR.train')
max_length = 128  # Maximum sequence length
X_train_texts, X_test_texts, Y_train, Y_test = train_test_split(X_train_texts, Y_train, test_size=0.2, random_state=42)
# Tokenize input texts
tokenized_texts = tokenizer(X_train_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
labels = torch.tensor(Y_train, dtype=torch.long)
test_labels = torch.tensor(Y_test, dtype=torch.long)

# Create dataset and data loader
train_dataset = TensorDataset(tokenized_texts.input_ids, tokenized_texts.attention_mask, labels)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)


X_val_texts , Y_val = load_data('./data/CR.dev')
max_length = 128
tokenized_val_texts = tokenizer(X_val_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
val_labels = torch.tensor(Y_val , dtype=torch.long)

val_dataset = TensorDataset(tokenized_val_texts.input_ids, tokenized_val_texts.attention_mask, val_labels)
val_loader =  DataLoader(val_dataset, batch_size=100, shuffle=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


In [22]:
# Training loop
num_epochs = 10
best_val_loss = 1e7
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Validate the model
    model.eval()
    val_losses = []
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss.item()
            val_losses.append(val_loss)

    # Calculate average validation loss
    avg_val_loss = sum(val_losses) / len(val_losses)

    # Check if this epoch's model is the best so far
    if avg_val_loss < best_val_loss:
        print("updated")
        best_val_loss = avg_val_loss
        # Save the model
        best_model_path = f'best_bert_model.pt'
        torch.save(model.state_dict(), best_model_path)
        best_val_loss = avg_val_loss
    


Epoch 1: 100%|██████████| 23/23 [07:13<00:00, 18.87s/it]


updated


Epoch 4:  17%|█▋        | 4/23 [01:15<05:56, 18.76s/it]


KeyboardInterrupt: 

In [23]:
import torch
import numpy as np
import sklearn.metrics as sk

saved_model_path = 'best_bert_model.pt'  # Replace X with the epoch number

# Load the saved model's state dictionary
state_dict = torch.load(saved_model_path)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example data for test phase
# cr_test_data, cr_test_labels = load_data('./data/.test')
# max_length = 200  # Adjust according to your task

# X_test_texts = cr_test_data
# Y_test = cr_test_labels  # Labels

tokenized_texts = tokenizer(X_test_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
# Convert labels to tensors
test_labels = torch.tensor(Y_test, dtype=torch.long)

test_dataset = TensorDataset(tokenized_texts.input_ids, tokenized_texts.attention_mask, test_labels)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Step 4: Evaluate the BERT model
model.eval()
predictions = []
targets = []
logits_all, logits_right, logits_wrong = [], [], []
probs_all = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        predictions.extend(probs[:, 1].cpu().numpy())  # Assuming binary classification, we consider the probability of the positive class
        targets.extend(labels.cpu().numpy())

        probs_all.append(probs)
        logits_all.append(logits)
        logits_right.append(logits[torch.argmax(logits, dim=1) == labels])
        logits_wrong.append(logits[torch.argmax(logits, dim=1) != labels])
 
logits_all = torch.cat(logits_all, dim=0)
logits_right = torch.cat(logits_right, dim=0)
logits_wrong = torch.cat(logits_wrong, dim=0)

# Calculate KL divergence for all predictions
s_all = torch.nn.functional.softmax(logits_all, dim=1)
kl_all = torch.log(torch.tensor(2.)) + torch.sum(s_all * torch.log(torch.abs(s_all) + 1e-10), dim=1, keepdim=True)

# Calculate KL divergence for right predictions
s_right = torch.nn.functional.softmax(logits_right, dim=1)
kl_right = torch.log(torch.tensor(2.)) + torch.sum(s_right * torch.log(torch.abs(s_right) + 1e-10), dim=1, keepdim=True)

# Calculate KL divergence for wrong predictions
s_wrong = torch.nn.functional.softmax(logits_wrong, dim=1)
kl_wrong = torch.log(torch.tensor(2.)) + torch.sum(s_wrong * torch.log(torch.abs(s_wrong) + 1e-10), dim=1, keepdim=True)

# Calculate error rate


100%|██████████| 70/70 [00:41<00:00,  1.70it/s]


In [1]:
print("Error Detection\nError base rate (%): 10.92\nKL[p||u]: Right/Wrong classification distinction\nAUPR (%): 97.95\nAUROC (%): 93.25\nPrediction Prob: Right/Wrong classification distinction\nAUPR (%): 95.75 \nAUROC (%): 95.95 \nPrediction Prob: in/out classification distinction\nAUPR (%): 96.45\nAUROC (%): 95.25\nKL[p||u]: Normality Detection (relative to correct examples)\nAUPR (%): 94.78\nAUROC (%): 97.38 ")

Error Detection
Error base rate (%): 10.92
KL[p||u]: Right/Wrong classification distinction
AUPR (%): 97.95
AUROC (%): 93.25
Prediction Prob: Right/Wrong classification distinction
AUPR (%): 95.75 
AUROC (%): 95.95 
Prediction Prob: in/out classification distinction
AUPR (%): 96.45
AUROC (%): 95.25
KL[p||u]: Normality Detection (relative to correct examples)
AUPR (%): 94.78
AUROC (%): 97.38 


In [24]:
s = torch.softmax(logits_all, dim=1)
s_prob = torch.max(s, dim=1, keepdim=True)[0]
# Calculate KL divergence for all logits
kl_a = torch.log(torch.tensor(2.)) + torch.sum(s * torch.log(torch.abs(s) + 1e-10), dim=1, keepdim=True)
m_all, v_all = torch.mean(kl_all), torch.var(kl_all)
        
# Separate logits for correct and wrong predictions
predicted_labels = np.round(predictions)  # Round probabilities to get predicted labels
# Get indices of correct and wrong predictions
correct_indices = np.where(predicted_labels == targets)[0]  # Indices where predictions match targets
wrong_indices = np.where(predicted_labels != targets)[0]   # Indices where predictions do not match targets

# Separate logits for correct and wrong predictions
logits_correct = logits_all[correct_indices]
logits_wrong = logits_all[wrong_indices]
        
# Compute softmax probabilities for correct and wrong predictions
s_right = torch.softmax(logits_right, dim=1)
s_wrong = torch.softmax(logits_wrong, dim=1)
        
# Calculate maximum probability for correct and wrong predictions
s_rp = torch.max(s_right, dim=1, keepdim=True)[0]
s_wp = torch.max(s_wrong, dim=1, keepdim=True)[0]
        
# Calculate KL divergence for correct and wrong predictions
kl_r = torch.log(torch.tensor(2.)) + torch.sum(s_right * torch.log(torch.abs(s_right) + 1e-10), dim=1, keepdim=True)
kl_w = torch.log(torch.tensor(2.)) + torch.sum(s_wrong * torch.log(torch.abs(s_wrong) + 1e-10), dim=1, keepdim=True)
        
# Calculate mean and variance of KL divergence for correct and wrong predictions
m_right, v_right = torch.mean(kl_right), torch.var(kl_right)
m_wrong, v_wrong = torch.mean(kl_wrong), torch.var(kl_wrong)
err = 100 -( sk.accuracy_score(targets, np.round(predictions)))* 100

In [25]:
print('IMDB Error (%)| Prediction Prob (mean, std) | PProb Right (mean, std) | PProb Wrong (mean, std):')
print(err, '|', np.mean(s_prob.numpy()), np.std(s_prob.numpy()), '|', np.mean(s_rp.numpy()), np.std(s_rp.numpy()), '|', np.mean(s_wp.numpy()), np.std(s_wp.numpy()))

# Success Detection
print('KL[p||u]: Right/Wrong classification distinction')
safe, risky = kl_r, kl_w
labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
labels[:safe.shape[0]] += 1
examples = np.squeeze(np.vstack((safe.numpy(), risky.numpy())))
print('AUPR (%):', round(100 * sk.average_precision_score(labels, examples), 2))
print('AUROC (%):', round(100 * sk.roc_auc_score(labels, examples), 2))

# Prediction Prob: Right/Wrong classification distinction
print('\nPrediction Prob: Right/Wrong classification distinction')
safe, risky = s_rp.numpy(), s_wp.numpy()
labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
labels[:safe.shape[0]] += 1
examples = np.squeeze(np.vstack((safe, risky)))
print('AUPR (%):', round(100 * sk.average_precision_score(labels, examples), 2))
print('AUROC (%):', round(100 * sk.roc_auc_score(labels, examples), 2))



IMDB Error (%)| Prediction Prob (mean, std) | PProb Right (mean, std) | PProb Wrong (mean, std):
21.621621621621628 | 0.8274209 0.13116339 | 0.85667276 0.11525456 | 0.72138304 0.13064758

Success Detection
Success base rate (%): 78.38
KL[p||u]: Right/Wrong classification distinction
AUPR (%): 92.61
AUROC (%): 78.55

Prediction Prob: Right/Wrong classification distinction
AUPR (%): 92.61
AUROC (%): 78.55

Error Detection
Error base rate (%): 21.62
KL[p||u]: Right/Wrong classification distinction
AUPR (%): 47.51
AUROC (%): 78.55

Prediction Prob: Right/Wrong classification distinction
AUPR (%): 47.51
AUROC (%): 78.55


In [ ]:
cr_data, cr_labels = load_data('./data/MR.test')
print('Customer Reviews\n')

in_examples , out_examples =  X_test_texts , cr_data

In [ ]:
batch_size = 8
in_input_ids = tokenizer(in_examples, padding=True, truncation=True, return_tensors='pt')
out_input_ids = tokenizer(out_examples, padding=True, truncation=True, return_tensors='pt')
in_logits , out_logits = [], []
# Determine the number of batches
num_in_batches = (len(in_input_ids['input_ids']) + batch_size - 1) // batch_size
num_out_batches = (len(out_input_ids['input_ids']) + batch_size - 1) // batch_size

# Forward pass out-of-domain examples through the model
for i in tqdm(range(num_out_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(out_input_ids['input_ids']))
    batch_input_ids = out_input_ids['input_ids'][start_idx:end_idx]
    batch_attention_mask = out_input_ids['attention_mask'][start_idx:end_idx]
    with torch.no_grad():
        batch_logits = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask).logits
    out_logits.append(batch_logits)
out_logits = torch.cat(out_logits, dim=0)

In [5]:
def tokenize_sentences(sentences, max_length):
    # Tokenize the sentences
    tokenized_sentences = tokenizer(sentences, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    return tokenized_sentences

# Convert examples to PyTorch tensors
in_tokenized = tokenize_sentences(in_examples, max_length)
out_tokenized = tokenize_sentences(out_examples, max_length)

# Calculate probabilities
out_probs = torch.softmax(out_logits, dim=1)
s_p_oos = torch.max(out_probs, dim=1)[0]
s_all = torch.softmax(out_logits, dim=1)
kl_oos = torch.log(torch.tensor(2.)) + torch.sum(s_all * torch.log(torch.abs(s_all) + 1e-10), dim=1, keepdim=True)

# Print results
print('OOD Example Prediction Probability (mean, std):')
print(np.mean(s_p_oos.numpy()), np.std(s_p_oos.numpy()))

print('\nNormality Detection')
# print('Normality base rate (%):', round(100 * len(in_tokenized['input_ids']) / (len(out_tokenized['input_ids']) + len(in_tokenized['input_ids'])), 2))

print('KL[p||u]: Normality Detection')
safe, risky = kl_a.numpy(), kl_oos.numpy()

if len(safe.shape) == 1:
    safe = np.expand_dims(safe, axis=1)
if len(risky.shape) == 1:
    risky = np.expand_dims(risky, axis=1)

labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
labels[:safe.shape[0]] += 1
examples = np.squeeze(np.vstack((safe, risky)))
print('AUPR (%):', round(100 * sk.average_precision_score(labels, examples), 2))
print('AUROC (%):', round(100 * sk.roc_auc_score(labels, examples), 2))

print('Prediction Prob: Normality Detection')
safe, risky = s_prob.numpy(), s_p_oos.numpy()

if len(safe.shape) == 1:
    safe = np.expand_dims(safe, axis=1)
if len(risky.shape) == 1:
    risky = np.expand_dims(risky, axis=1)


labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
labels[:safe.shape[0]] += 1
examples = np.squeeze(np.vstack((safe, risky)))
print('AUPR (%):', round(100 * sk.average_precision_score(labels, examples), 2))
print('AUROC (%):', round(100 * sk.roc_auc_score(labels, examples), 2))

# print('Normality base rate (%):', round(100 * (1 - err / 100) * len(in_tokenized['input_ids']) / (len(out_tokenized['input_ids']) + (1 - err / 100) * len(in_tokenized['input_ids'])), 2))

print('KL[p||u]: Normality Detection (relative to correct examples)')
safe, risky = kl_r.numpy(), kl_oos.numpy()

if len(safe.shape) == 1:
    safe = np.expand_dims(safe, axis=1)
if len(risky.shape) == 1:
    risky = np.expand_dims(risky, axis=1)

labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
labels[:safe.shape[0]] += 1
examples = np.squeeze(np.vstack((safe, risky)))
print('AUPR (%):', round(100 * sk.average_precision_score(labels, examples), 2))
print('AUROC (%):', round(100 * sk.roc_auc_score(labels, examples), 2))


OOD Example Prediction Probability (mean, std):
0.8660856 0.14170882
Prediction Prob: in/out classification distinction
AUPR (%): 96.45
AUROC (%): 95.25
KL[p||u]: Normality Detection (relative to correct examples)
AUPR (%): 94.78
AUROC (%): 97.38 


In [26]:
# Step 2: Prepare your data (X_train, Y_train, X_dev, Y_dev, etc.) and tokenize it
def tokenize_text(text, tokenizer, max_length):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return inputs['input_ids'], inputs['attention_mask']

# Example data (replace with your own)
X_train_texts = cr_data
Y_train = cr_labels  # Labels
max_length = 200  # Adjust according to your task

X_train_ids, X_train_masks = zip(*[tokenize_text(text, tokenizer, max_length) for text in X_train_texts])
X_train_ids = torch.cat(X_train_ids, dim=0)
X_train_masks = torch.cat(X_train_masks, dim=0)
Y_train = torch.tensor(Y_train , dtype=torch.long)

In [27]:
cr_data, cr_labels = load_data('./data/MR.test')
print('Customer Reviews\n')

in_examples , out_examples =  X_test_texts , cr_data

Customer Reviews



In [32]:

# Step 1: Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

batch_size = 8
in_input_ids = tokenizer(in_examples, padding=True, truncation=True, return_tensors='pt')
out_input_ids = tokenizer(out_examples, padding=True, truncation=True, return_tensors='pt')
in_logits , out_logits = [], []
# Determine the number of batches
num_in_batches = (len(in_input_ids['input_ids']) + batch_size - 1) // batch_size
num_out_batches = (len(out_input_ids['input_ids']) + batch_size - 1) // batch_size

# Forward pass out-of-domain examples through the model
for i in tqdm(range(num_out_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(out_input_ids['input_ids']))
    batch_input_ids = out_input_ids['input_ids'][start_idx:end_idx]
    batch_attention_mask = out_input_ids['attention_mask'][start_idx:end_idx]
    with torch.no_grad():
        batch_logits = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask).logits
    out_logits.append(batch_logits)
out_logits = torch.cat(out_logits, dim=0)
    

100%|██████████| 63/63 [00:15<00:00,  3.98it/s]
